In [1]:
# packages
import requests
from bs4 import BeautifulSoup
from itertools import zip_longest
import csv
import re
from textblob import TextBlob
from textblob_ar import TextBlob
from textblob_ar.correction import TextCorrection

In [3]:
Author = []
title = []
links = []
book = []
language = []
publishing_house = []
size = []
format = []
category = []
page_num = 1
numbers = []
no_of_pages = []
page_n = []
brief = []  # في رغيف

main = "https://www.arab-books.com/"    # The Main Link Of The Website

result_main = requests.get(main)        # Request The Link

src = result_main.content               # Load Website Content (HTML - JS - ...)
# print(src)

# parse content
soup = BeautifulSoup(src, 'lxml')       # Pass Website Content To Parsser
# print(soup)

page = soup.find_all("a", {"class":"page-numbers"})  # Get All <a> Tags That Its Class Is "page-numbers"

for i in page :
    page_n.append(i.text)               # Get The Text from "page-numbers"

web_pages = int(page_n[2])              # The Number Of Last Page

#print(web_pages)

##### auto correct
def correct(word):
    ret = ""
    blob = TextBlob(word)               # Tokenize
    print(blob.tokens)
    for token in blob.tokens:           # Iterate over all tokens
        ret += TextCorrection().correction(token, top=True) +" "    # Correct Misspelled Tokens Then Append it
    return ret

x = 5
while page_num <= web_pages:           # Iterate over all Web Pages
#     if x == 0:
#         break
#     x-=1
    # target URL
    url = f"https://www.arab-books.com//page/{page_num}"    # Update URL to go to next page

    result = requests.get(url)

    src = result.content
    #print(src)


    # parse content
    soup = BeautifulSoup(src, 'lxml')
    #print(soup)


    Authors = soup.find_all("div", {"class":"book-writer"})
    #print(Authors)

    titles = soup.find_all("div", {"class":"excerpt-book"})
    #print(titles)



    for i in range(len(Authors)):
        if Authors[i].text.replace("\n", "") == 'الكاتب هارون يحيىالكاتب هارون يحيى':
            Author.append("هارون يحيى")
        else:
            Author.append(Authors[i].text)
        title.append(titles[i].text)
        links.append(titles[i].find("a").attrs["href"])

    # Correct And Remove 
    Author = [s.replace("\n", "") for s in Author]
    Author = [s.replace("الكاتب ", "") for s in Author]
    Author = [correct(s) for s in Author]       # Correct
    title = [s.replace("PDF", "") for s in title]
    title = [s.replace("\n", "") for s in title]
    title = [correct(s) for s in title]         # Correct
    
    
    #print(links)
    #print(Author)
    #print(title)

    page_num +=1
    if page_num == 111 :       # Contain Broken Link
        page_num = page_num + 1

    print("page switched")


for link in links:            # Iterate over all Links of The Books
    book = {}
    result = requests.get(link)
    src = result.content
    soup = BeautifulSoup(src,"lxml")

    descrip = soup.find("div", {"class":"book-infos"}).ul     
    content = soup.find("div", {"id":"books-content"}) 
    
    
    infos = list(soup.find("div", class_='book-infos').stripped_strings)    # for other information about the book


#     print(content)
    
    for info in infos:
        if info == 'لغة الكتاب:':
            book["language"] = infos[infos.index(info) + 1]
        if info == 'دار النشر:':
            book["publishing house"] = infos[infos.index(info) + 1]
        if info == 'حجم الكتاب:':
            book["size"] = infos[infos.index(info) + 1]
        if info == 'ملف الكتاب:':
            book["format"] = infos[infos.index(info) + 1]
        if info == 'قسم الكتاب:':
            book["category"] = infos[infos.index(info) + 1][6:]


    # Handle Empty Fields
    if book["language"] == "عدد الصّفحات:":
        language.append(" ")
    else:
        language.append(correct(book["language"]))    # Correct


    if book["publishing house"] == "حجم الكتاب:":
        publishing_house.append(" ")
    else:
        publishing_house.append(correct(book["publishing house"])) # Correct


    size.append(book["size"])


    if book["format"] == "تبليغ حقوق الملكية:":
        format.append(" ")
    else:
        format.append(book["format"])


    if book["category"] == "كتاب:":
        category.append(" ")
    else:
        category.append(Correct(book["category"])) # Correct


#print(language)
#print(publishing_house)
#print(size)
#print(format)
#print(category)



    paragraphs = content.find_all(['p', 'h4', 'h3' , 'h2' ,'li'])
    res = ""
    for pr in range(len(paragraphs)):
        if(str(paragraphs[pr]).find("ملخص") != -1 or str(paragraphs[pr]).find("تلخيص") != -1):
            pr += 1
            if(pr < len(paragraphs) and str(paragraphs[pr]).find("تحميل رواية عربي") != -1):
                pr += 1
            if(pr < len(paragraphs) and str(paragraphs[pr]).find("الروايات والكتب العربية تعتبر من الروابط بيننا") != -1):
                pr += 1
            if(pr < len(paragraphs) and str(paragraphs[pr].find("تحميل كتب عربي") != -1)):
                pr += 1
            while True:
                if pr >= len(paragraphs) or str(paragraphs[pr]).find("نبذة عن كاتب") != -1:
                    break
                res+= str(paragraphs[pr]) + " "
                pr += 1
    res = re.sub(r'[^(\u0621-\u064A\u0660-\u0669 | \s)]', '',str(res))  # [^(\u0621-\u064A\u0660-\u0669 | \s)] Remove Any Character Not Arabic
    res = re.sub(r'\s+', ' ',str(res))
    res = Correct(res)   # Correct
    
    brief.append(res)
#     print(res)
#     print("---------------------------------------------------")
    


    desc_text = ""
    for li in descrip.find_all("li"):
        desc_text += li.text + "/ "
    desc_text = desc_text[:-2]   # remove '/' from last text
    numbers.append(re.findall(r'\d{1,3}',desc_text))
    
#print(numbers)

print(len(brief))

for m in numbers:
    if len(m) == 0:
        m.append(1)
        no_of_pages.append(m[0])
    else :
        no_of_pages.append(m[0])

#print(no_of_pages)

#print(len(no_of_pages))


file_list = [Author , title , category ,  links , no_of_pages , language ,publishing_house ,size ,format, brief]
imported = zip_longest(*file_list)

with open('scrapping.csv',"w" , encoding = 'utf-8') as project :
    wr = csv.writer(project)
    wr.writerow(["Author", "Title", "Category", "Link for PDF", "Number Of Pages", "Language" , "Publishing House", "Size", "Format", "Brief"])
    wr.writerows(imported)

page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page switched
page s